In [1]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from matplotlib import colors as mcolors
import pathlib
from typing import List, Tuple
import uproot3

In [2]:
input_file_name = 'Lb2JPsiL_Run2_Sim_2016_MagUp_v12_VertexingDebug.root'
input_dir = '/scratch/user/adegenna/'

input_VF = input_dir + 'top50_standard/' + input_file_name
input_X = input_dir + 'top50_all_xrescaled/' + input_file_name
input_Y = input_dir + 'top50_all_yrescaled/' + input_file_name
input_Z = input_dir + 'top50_all_zrescaled/' + input_file_name

In [3]:
save_plots = False
black_and_white = True
show_titles = True
thick_hist_lines = False

## Pick what extra branches you want
with_dtf = True
with_pidsubs = False
with_protoparticles = True
with_2Ddebug = True
with_truevtx_kinematics = True

## Perform analysis with:
## - All data: 'b' (both)
## - Only Lambdas: 'm' (matter)
## - Only antiLambdas: 'a' (antimatter)
use_matter_antimatter = 'b'

In [4]:
if use_matter_antimatter == 'b':
    proton_symbol = 'p'
    pion_symbol = '\pi'
    file_suffix = ''
elif use_matter_antimatter == 'm':
    proton_symbol = 'p'
    pion_symbol = '\pi^-'
    file_suffix = '_matter'
elif use_matter_antimatter == 'a':
    proton_symbol = '\bar{p}'
    pion_symbol = '\pi^+'
    file_suffix = '_antimatter'
else:
    raise ValueError

In [5]:
## Create plots directory, if missing
plots_dir = (str(pathlib.Path.home())
                  + '/vertex-reconstruction-studies/'
                  + 'plots/'
                  + 'E_studies_on_integrated_root_files/'
                  + 'sigma_xyz_cross-comparison/'
)
pathlib.Path(plots_dir).mkdir(parents=True, exist_ok=True)

# Data import quasi-class (v.3.0)

In [6]:
def root_file_to_df(
    filename: str,
    branches_ordinary: List[str] = [],
    branches_jagged: List[str] = [],   ## These are the protoparticles, really
    truth_tree: bool = False,
    slice_ordinary: bool = True
) -> pd.DataFrame:
    if len(branches_jagged) == 0:
        have_jagged = False
    else:
        have_jagged = True
    
    if len(branches_ordinary) == 0:
        have_ordinary = False
    else:
        have_ordinary = True
    
    with uproot3.open(filename) as file_root:
        if truth_tree:
            tree = file_root['Lb_MC/MCDecayTree']
        else:
            tree = file_root['Lb_T/DecayTree']
        ## Load dataFrame with desired branches
        if have_ordinary:
            if slice_ordinary:
                df_ord = tree.pandas.df(branches_ordinary).loc[(slice(None), 0), :]
            else:
                df_ord = tree.pandas.df(branches_ordinary)
        if have_jagged:
            df_jag = tree.pandas.df(branches_jagged, flatten=False)
            for col in list(df_jag):
                df_jag[col] = df_jag[col].str[0]
        
        if have_ordinary and not have_jagged:
            df = df_ord
        elif not have_ordinary and have_jagged:
            df = df_jag
        elif have_ordinary and have_jagged:
            df = pd.merge(df_ord, df_jag, left_index=True, right_index=True)
            if len(df) != len(df_jag):
                raise ValueError
        else:
            raise ValueError
            
        ## Reset indices
        df.reset_index(drop=True, inplace=True)
        return df

## Branches
### Choose branches

In [7]:
composite_particles = ['Lb', 'L', 'Jpsi']
basic_particles = ['p', 'pim', 'mum', 'mup']
lambda_baryons = ['L', 'Lb']

lambdab_daughters = ['L', 'Jpsi']
lambdab_daughters_DTF = ['Lambda0', 'J_psi_1S']

lambdab_nephews = {
    'L': ['p', 'pim'],
    'Jpsi': ['mum', 'mup'],
}
lambdab_nephews_DTF = {
    'Lambda0': ['pplus', 'piplus'],
    'J_psi_1S': ['muminus_0', 'muminus'],
}

DTF_mass_constraints = ['FixJPsi', 'FixJPsiLambda']

#### Original names

In [8]:
## Reco branches
reco_branches = []
for prefix in composite_particles:
    for component in ['X', 'Y', 'Z']:
        reco_branches.append(prefix + '_ENDVERTEX_' + component)
    reco_branches.append(prefix + '_M')
    reco_branches.append(prefix + '_MM')
        
for prefix in composite_particles + basic_particles:
    for component in ['X', 'Y', 'Z']:
        reco_branches.append(prefix + '_P' + component)
    reco_branches.append(prefix + '_ID')    
    
for prefix in lambda_baryons:
    reco_branches.append(prefix + '_BPVDIRA')
    reco_branches.append(prefix + '_BPVIPCHI2')
    reco_branches.append(prefix + '_BPVVDCHI2')
    reco_branches.append(prefix + '_VFASPF_CHI2_VDOF')
    
for feature in ['X', 'Y', 'Z', 'CHI2', 'NDOF']:
    reco_branches.append('Lb_OWNPV_' + feature)

reco_branches.append('Polarity')

## Truth branches
truth_branches = []

for prefix in lambda_baryons:
    for component in ['X', 'Y', 'Z']:
        truth_branches.append(prefix + '_TRUEENDVERTEX_' + component)
        
for prefix in basic_particles:
    for component in ['X', 'Y', 'Z']:
        truth_branches.append(prefix + '_TRUEP_' + component)

truth_branches.append('L_TRUEID')
truth_branches.append('Lb_TRUEID')
        
for prefix in basic_particles + composite_particles:
    truth_branches.append(prefix + '_Reconstructible')
    truth_branches.append(prefix + '_Reconstructed')

## DTF branches
dtf_branches = []
if with_dtf:
    for mass_constraint in DTF_mass_constraints:
        dtf_branches.append('Lb_DTF_' + mass_constraint + '_status')
        dtf_branches.append('Lb_DTF_' + mass_constraint + '_M')
        dtf_branches.append('Lb_DTF_' + mass_constraint + '_P')
        dtf_branches.append('Lb_DTF_' + mass_constraint + '_chi2')
        dtf_branches.append('Lb_DTF_' + mass_constraint + '_nDOF')
        for feature in ['key', 'X', 'Y', 'Z']:
            dtf_branches.append('Lb_DTF_' + mass_constraint + '_PV_' + feature)

        for mother in lambdab_daughters_DTF:
            dtf_branches.append('Lb_DTF_' + mass_constraint + '_' + mother + '_M')
            dtf_branches.append('Lb_DTF_' + mass_constraint + '_' + mother + '_P')
            dtf_branches.append('Lb_DTF_' + mass_constraint + '_' + mother + '_decayLength')

            for daughter in lambdab_nephews_DTF[mother]:
                for component in ['X', 'Y', 'Z']:
                    dtf_branches.append('Lb_DTF_' + mass_constraint + '_' + mother + '_' + daughter + '_P' + component)

## PIDsubs branches
pidsubs_branches = []
if with_pidsubs:
    for mass_constraint in DTF_mass_constraints:
        pidsubs_branches.append('Lb_DTF_' + mass_constraint + '_PIDSubs' + '_status')
        pidsubs_branches.append('Lb_DTF_' + mass_constraint + '_PIDSubs' + '_M')
        pidsubs_branches.append('Lb_DTF_' + mass_constraint + '_PIDSubs' + '_P')
        pidsubs_branches.append('Lb_DTF_' + mass_constraint + '_PIDSubs' + '_chi2')
        pidsubs_branches.append('Lb_DTF_' + mass_constraint + '_PIDSubs' + '_nDOF')
        for feature in ['key', 'X', 'Y', 'Z']:
            pidsubs_branches.append('Lb_DTF_' + mass_constraint + '_PIDSubs' + '_PV_' + feature)

        for mother in lambdab_daughters_DTF:
            pidsubs_branches.append('Lb_DTF_' + mass_constraint + '_PIDSubs' + '_' + mother + '_M')
            pidsubs_branches.append('Lb_DTF_' + mass_constraint + '_PIDSubs' + '_' + mother + '_P')
            pidsubs_branches.append('Lb_DTF_' + mass_constraint + '_PIDSubs' + '_' + mother + '_decayLength')

            for daughter in lambdab_nephews_DTF[mother]:
                for component in ['X', 'Y', 'Z']:
                    pidsubs_branches.append('Lb_DTF_' + mass_constraint + '_PIDSubs' + '_' + mother + '_' + daughter + '_P' + component)
                    
    pidsubs_branches.append('TrackType')

## Protoparticle branches
pp_branches = []
if with_protoparticles:
    for part in ['p', 'pim']:
        for feature in ['', 'P']:
            for comp in ['X', 'Y', 'Z']:
                pp_branches.append(part + '_PP_' + feature + comp)
        
        pmc_components = [0, 1, 3, 4, 5] ## 2 is z, all zeros
        
        for row in pmc_components:
            for column in [c for c in pmc_components if c >= row]:
                pp_branches.append(part + '_PP_POSMOMCOV' + '_' +  str(row) +  '_' + str(column))
                
## 2D debug branches
debug_branches = []
if with_2Ddebug:
    debug_branches.append('L_ENDVERTEX_XERR')
    debug_branches.append('L_ENDVERTEX_YERR')
    debug_branches.append('L_ENDVERTEX_ZERR')
    debug_branches.append('L_ALGO_ID')
    
## True vertex kinematics
truevtxkin_branches = []
if with_truevtx_kinematics:
    for kinematic_computation_point in ['AtTrueVtx', 'AtFirstMeas']:
        for prefix in ['p', 'pim']:
            ## Actually no need for z, it's fixed by the transporter
            for component in ['X', 'Y', 'Z']:
                truevtxkin_branches.append(prefix + '_' + kinematic_computation_point + '_REFP_' + component)
                truevtxkin_branches.append(prefix + '_' + kinematic_computation_point + '_REFP_' + component + "ERR")

            for component in ['X', 'Y', 'Z']:
                truevtxkin_branches.append(prefix + '_' + kinematic_computation_point + '_P_' + component)
                truevtxkin_branches.append(prefix + '_' + kinematic_computation_point + '_P_' + component + "ERR")
                
            for component in ['X', 'Y', 'Z']:
                truevtxkin_branches.append('MCTRUTH_' + prefix + '_' + kinematic_computation_point + '_REFP_' + component)
                truevtxkin_branches.append('MCTRUTH_' + prefix + '_' + kinematic_computation_point + '_REFP_' + component + "ERR")

            for component in ['X', 'Y', 'Z']:
                truevtxkin_branches.append('MCTRUTH_' + prefix + '_' + kinematic_computation_point + '_P_' + component)
                truevtxkin_branches.append('MCTRUTH_' + prefix + '_' + kinematic_computation_point + '_P_' + component + "ERR")
    
branches = (
    reco_branches + truth_branches
    + dtf_branches + pidsubs_branches
    + debug_branches + truevtxkin_branches
)

branches_jagged = pp_branches

#### New names

In [9]:
## Reco branches
reco_branches_renamed = []
for prefix in composite_particles:
    for component in ['X', 'Y', 'Z']:
        reco_branches_renamed.append(prefix + '_ENDVERTEX_' + component)
    reco_branches_renamed.append(prefix + '_M')
    reco_branches_renamed.append(prefix + '_MM')
        
for prefix in composite_particles + basic_particles:
    for component in ['X', 'Y', 'Z']:
        reco_branches_renamed.append(prefix + '_P' + component)
    reco_branches_renamed.append(prefix + '_ID')    
    
for prefix in lambda_baryons:
    reco_branches_renamed.append(prefix + '_BPVDIRA')
    reco_branches_renamed.append(prefix + '_BPVIPCHI2')
    reco_branches_renamed.append(prefix + '_BPVVDCHI2')
    reco_branches_renamed.append(prefix + '_VFASPF_CHI2_VDOF')
    
for feature in ['X', 'Y', 'Z', 'CHI2', 'NDOF']:
    reco_branches_renamed.append('Lb_OWNPV_' + feature)

reco_branches_renamed.append('Polarity')

## Truth branches
truth_branches_renamed = []

for prefix in lambda_baryons:
    for component in ['X', 'Y', 'Z']:
        truth_branches_renamed.append('MCTRUTH_' + prefix + '_ENDVERTEX_' + component)
        
for prefix in basic_particles:
    for component in ['X', 'Y', 'Z']:
        truth_branches_renamed.append('MCTRUTH_' + prefix + '_P' + component)

truth_branches_renamed.append('MCTRUTH_L_ID')
truth_branches_renamed.append('MCTRUTH_Lb_ID')
        
for prefix in basic_particles + composite_particles:
    truth_branches_renamed.append('MCTRUTH_' + prefix + '_Reconstructible')
    truth_branches_renamed.append('MCTRUTH_' + prefix + '_Reconstructed')

## DTF branches
dtf_branches_renamed = []
if with_dtf:
    for mass_constraint in DTF_mass_constraints:
        dtf_branches_renamed.append('DTF_' + mass_constraint + '_status')
        dtf_branches_renamed.append('DTF_' + mass_constraint + '_Lb_M')
        dtf_branches_renamed.append('DTF_' + mass_constraint + '_Lb_P')
        dtf_branches_renamed.append('DTF_' + mass_constraint + '_chi2')
        dtf_branches_renamed.append('DTF_' + mass_constraint + '_nDOF')
        for feature in ['key', 'X', 'Y', 'Z']:
            dtf_branches_renamed.append('Lb_DTF_' + mass_constraint + '_PV_' + feature)

        for mother in lambdab_daughters:
            dtf_branches_renamed.append('DTF_' + mass_constraint + '_' + mother + '_M')
            dtf_branches_renamed.append('DTF_' + mass_constraint + '_' + mother + '_P')
            dtf_branches_renamed.append('DTF_' + mass_constraint + '_' + mother + '_decayLength')

            for daughter in lambdab_nephews[mother]:
                for component in ['X', 'Y', 'Z']:
                    dtf_branches_renamed.append('DTF_' + mass_constraint + '_' + daughter + '_P' + component)

## PIDsubs branches
pidsubs_branches_renamed = []
if with_pidsubs:
    for mass_constraint in DTF_mass_constraints:
        pidsubs_branches_renamed.append('DTF_' + mass_constraint + '_PIDSubs' + '_status')
        pidsubs_branches_renamed.append('DTF_' + mass_constraint + '_PIDSubs' + '_Lb_M')
        pidsubs_branches_renamed.append('DTF_' + mass_constraint + '_PIDSubs' + '_Lb_P')
        pidsubs_branches_renamed.append('DTF_' + mass_constraint + '_PIDSubs' + '_chi2')
        pidsubs_branches_renamed.append('DTF_' + mass_constraint + '_PIDSubs' + '_nDOF')
        for feature in ['key', 'X', 'Y', 'Z']:
            pidsubs_branches_renamed.append('DTF_' + mass_constraint + '_PIDSubs' + '_PV_' + feature)

        for mother in lambdab_daughters:
            pidsubs_branches_renamed.append('DTF_' + mass_constraint + '_PIDSubs' + '_' + mother + '_M')
            pidsubs_branches_renamed.append('DTF_' + mass_constraint + '_PIDSubs' + '_' + mother + '_P')
            pidsubs_branches_renamed.append('DTF_' + mass_constraint + '_PIDSubs' + '_' + mother + '_decayLength')

            for daughter in lambdab_nephews[mother]:
                for component in ['X', 'Y', 'Z']:
                    pidsubs_branches_renamed.append('DTF_' + mass_constraint + '_PIDSubs' + '_' + daughter + '_P' + component)
                    
    pidsubs_branches_renamed.append('TrackType')

## Protoparticle branches
pp_branches_renamed = []
if with_protoparticles:
    for part in ['p', 'pim']:
        for feature in ['', 'P']:
            for comp in ['X', 'Y', 'Z']:
                pp_branches_renamed.append(part + '_PP_' + feature + comp)
        
        pmc_components = [0, 1, 3, 4, 5] ## 2 is z, all zeros
        
        for row in pmc_components:
            for column in [c for c in pmc_components if c >= row]:
                pp_branches_renamed.append(part + '_PP_POSMOMCOV' + '_' +  str(row) +  '_' + str(column))
                
## 2D debug branches
debug_branches_renamed = []
if with_2Ddebug:
    debug_branches_renamed.append('L_ENDVERTEX_XERR')
    debug_branches_renamed.append('L_ENDVERTEX_YERR')
    debug_branches_renamed.append('L_ENDVERTEX_ZERR')
    debug_branches_renamed.append('L_ALGO_ID')

## True vertex kinematics
truevtxkin_branches_renamed = []
if with_truevtx_kinematics:
    for kinematic_computation_point in ['AtTrueVtx', 'AtFirstMeas']:
        for prefix in ['p', 'pim']:
            ## Actually no need for z, it's fixed by the transporter
            for component in ['X', 'Y', 'Z']:
                truevtxkin_branches_renamed.append(prefix + '_' + kinematic_computation_point + '_REFP_' + component)
                truevtxkin_branches_renamed.append(prefix + '_' + kinematic_computation_point + '_REFP_' + component + "ERR")

            for component in ['X', 'Y', 'Z']:
                truevtxkin_branches_renamed.append(prefix + '_' + kinematic_computation_point + '_P_' + component)
                truevtxkin_branches_renamed.append(prefix + '_' + kinematic_computation_point + '_P_' + component + "ERR")
    
            for component in ['X', 'Y', 'Z']:
                truevtxkin_branches_renamed.append('MCTRUTH_' + prefix + '_' + kinematic_computation_point + '_REFP_' + component)
                truevtxkin_branches_renamed.append('MCTRUTH_' + prefix + '_' + kinematic_computation_point + '_REFP_' + component + "ERR")

            for component in ['X', 'Y', 'Z']:
                truevtxkin_branches_renamed.append('MCTRUTH_' + prefix + '_' + kinematic_computation_point + '_P_' + component)
                truevtxkin_branches_renamed.append('MCTRUTH_' + prefix + '_' + kinematic_computation_point + '_P_' + component + "ERR")
                
branches_renamed = (
    reco_branches_renamed + truth_branches_renamed
    + dtf_branches_renamed + pidsubs_branches_renamed
    + debug_branches_renamed + truevtxkin_branches_renamed
)

branches_jagged_renamed = pp_branches_renamed

In [10]:
if len(branches_renamed) != len(branches):
    raise ValueError("Number of ordinary columns before and after renaming does not match.")
    
if len(branches_jagged_renamed) != len(branches_jagged):
    raise ValueError("Number of jagged columns before and after renaming does not match.")

### `ALGO_ID` mapper

In [11]:
algo_id_dictionary = {
    -10: 'NonConverged',
     -1: 'Unspecified',
      0: 'VertexFitter',
      1: 'XBlownUpAlgorithm',
      2: 'YBlownUpAlgorithm',
      3: 'XYBlownUpAlgorithm',
      4: 'ZBlownUpAlgorithm',
      5: 'XZBlownUpAlgorithm',
      6: 'YZBlownUpAlgorithm',
      7: 'XYZBlownUpAlgorithm',
     10: 'Generic2DAlgorithm',
}

## Processing function

In [12]:
def import_data(input_file: str):
    df = root_file_to_df(input_file, branches, branches_jagged)

    column_rename_dictionary = {}
    for old_name, new_name in zip(branches, branches_renamed):
        column_rename_dictionary[old_name] = new_name
    for old_name, new_name in zip(branches_jagged, branches_jagged_renamed):
        column_rename_dictionary[old_name] = new_name

    df.rename(columns=column_rename_dictionary, inplace=True)
    
    ## Replace ALGO_ID with intelligible labels
    df.replace({'L_ALGO_ID': algo_id_dictionary}, inplace=True)
    
    if not use_matter_antimatter == 'b':
        if use_matter_antimatter == 'm':
            Lb_ID_constraint = 5122
        elif use_matter_antimatter == 'a':
            Lb_ID_constraint = -5122
        else:
            raise ValueError

        df = df.loc[df['MCTRUTH_Lb_ID'] == Lb_ID_constraint]
        
    return df

# Import the DataFrames

In [13]:
df_VF = import_data(input_VF)
df_VF

,Lb_ENDVERTEX_X,Lb_ENDVERTEX_Y,Lb_ENDVERTEX_Z,Lb_M,Lb_MM,L_ENDVERTEX_X,L_ENDVERTEX_Y,L_ENDVERTEX_Z,L_M,L_MM,...,pim_PP_POSMOMCOV_1_1,pim_PP_POSMOMCOV_1_3,pim_PP_POSMOMCOV_1_4,pim_PP_POSMOMCOV_1_5,pim_PP_POSMOMCOV_3_3,pim_PP_POSMOMCOV_3_4,pim_PP_POSMOMCOV_3_5,pim_PP_POSMOMCOV_4_4,pim_PP_POSMOMCOV_4_5,pim_PP_POSMOMCOV_5_5
0,1.674753,0.053259,0.004623,5548.595304,5994.525949,461.983261,-78.247981,6504.222043,1098.518302,1116.608101,...,0.156104,0.054083,-1.250532,1.336292,135.629715,-9.136084,3142.555664,17.418783,-213.811951,7.316263e+04
1,0.880395,-0.130333,15.654301,4900.959051,5196.066164,209.545799,199.243538,4381.083975,1228.240710,1286.308889,...,1.845794,-4.761426,-4.757078,-16.119181,2562.330811,9.229958,9031.324219,43.331825,32.646484,3.185937e+04
2,0.893069,-0.210212,27.346294,5669.609293,5447.484208,84.953310,-116.199707,5557.096303,1196.546077,1204.210206,...,0.156974,-0.272185,-2.498910,9.883920,760.612305,745.142334,-28860.775391,787.374390,-28372.277344,1.099216e+06
3,0.636031,-0.090822,-65.799311,5666.024183,5523.351851,-207.686722,152.674077,7297.794399,1142.059990,1140.502451,...,0.171479,0.277545,-2.941218,-7.591522,503.966400,-231.546097,-13094.367188,201.608795,6041.502441,3.405917e+05
4,0.986496,-0.173897,54.366690,5458.097962,5344.372839,133.523208,3.904319,6612.102919,1103.525428,1113.284255,...,0.174056,0.364528,-2.574241,10.254116,495.785950,39.231781,13094.487305,70.691956,1017.335205,3.464277e+05
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4588,0.847599,-0.210711,4.450657,5676.249335,5634.826933,221.339809,-151.638516,4306.754048,1187.718713,1185.807004,...,4.293598,-41.613556,-8.433111,-125.821960,10386.176758,-773.569397,31718.019531,105.008934,-2364.728271,9.691066e+04
4589,1.244353,0.273491,26.617082,5791.405084,6034.812168,303.069644,143.432016,6927.412624,1159.904419,1176.455599,...,1.251937,-2.488792,-19.226019,-123.327477,909.107849,1038.543579,42415.023438,1501.186279,48571.347656,1.983485e+06
4590,0.771442,-0.198648,-45.869868,9423.309520,9199.005287,-114.713797,-98.392843,5366.717903,5211.399379,2499.554590,...,0.975354,-11.960934,-1.553255,54.025459,13881.547852,-4576.853027,-64122.265625,1564.207886,21157.076172,2.963137e+05
4591,0.421768,-0.947067,-73.637508,5397.810490,5126.501383,-192.066143,-297.004409,4468.714184,1100.837596,1124.614171,...,0.929874,-5.542767,-0.706738,-174.402390,1382.818115,-2486.728760,44215.328125,4699.990723,-79816.640625,1.418817e+06


In [14]:
df_X = import_data(input_X)
df_X

,Lb_ENDVERTEX_X,Lb_ENDVERTEX_Y,Lb_ENDVERTEX_Z,Lb_M,Lb_MM,L_ENDVERTEX_X,L_ENDVERTEX_Y,L_ENDVERTEX_Z,L_M,L_MM,...,pim_PP_POSMOMCOV_1_1,pim_PP_POSMOMCOV_1_3,pim_PP_POSMOMCOV_1_4,pim_PP_POSMOMCOV_1_5,pim_PP_POSMOMCOV_3_3,pim_PP_POSMOMCOV_3_4,pim_PP_POSMOMCOV_3_5,pim_PP_POSMOMCOV_4_4,pim_PP_POSMOMCOV_4_5,pim_PP_POSMOMCOV_5_5
0,0.880396,-0.130332,15.654310,4905.645844,5206.683291,209.815708,198.703981,4375.034351,1231.389962,1288.543001,...,1.845794,-4.761426,-4.757078,-16.119181,2562.330811,9.229958,9031.324219,43.331825,32.646484,3.185937e+04
1,0.893067,-0.210211,27.346294,5667.336115,5447.890772,84.950689,-116.173875,5555.983100,1196.685207,1203.973021,...,0.156974,-0.272185,-2.498910,9.883920,760.612305,745.142334,-28860.775391,787.374390,-28372.277344,1.099216e+06
2,0.605714,-0.203049,22.148035,5645.205401,5601.574674,-202.498747,18.725639,3762.349599,1179.755864,1134.066434,...,0.158962,-0.209039,-1.985822,-4.321545,1088.073242,209.480865,26939.328125,90.186363,5198.505859,6.694764e+05
3,0.986467,-0.173922,54.367574,5447.084894,5361.743921,134.051222,3.959282,6635.673166,1102.995870,1114.170884,...,0.174056,0.364528,-2.574241,10.254116,495.785950,39.231781,13094.487305,70.691956,1017.335205,3.464277e+05
4,0.964616,0.025096,-0.666939,-3369.226766,8121.009825,-134.013677,189.405439,3150.833245,1049.717279,1411.434807,...,3.806791,37.160542,-7.632927,92.395973,75195.250000,18212.148438,191457.765625,4514.575684,46385.707031,4.875446e+05
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4603,0.734722,-0.098736,-61.234969,5711.861139,5658.615375,-114.635257,-30.251622,4136.849269,1274.282362,1296.663216,...,0.333853,-4.867212,-3.991302,36.106815,12834.206055,900.435059,-98631.953125,126.835609,-6909.498047,7.582912e+05
4604,0.847599,-0.210711,4.450657,5686.871518,5649.120788,222.721900,-152.508973,4332.504288,1210.928174,1204.985859,...,4.293598,-41.613556,-8.433111,-125.821960,10386.176758,-773.569397,31718.019531,105.008934,-2364.728271,9.691066e+04
4605,1.244345,0.273479,26.617010,5827.524278,6042.236647,303.092783,143.444177,6928.034797,1161.295817,1177.146963,...,1.251937,-2.488792,-19.226019,-123.327477,909.107849,1038.543579,42415.023438,1501.186279,48571.347656,1.983485e+06
4606,0.771440,-0.198648,-45.869866,9389.705847,9195.445595,-115.076971,-98.456727,5376.297260,5198.342263,2505.916628,...,0.975354,-11.960934,-1.553255,54.025459,13881.547852,-4576.853027,-64122.265625,1564.207886,21157.076172,2.963137e+05


In [15]:
df_Y = import_data(input_Y)
df_Y

,Lb_ENDVERTEX_X,Lb_ENDVERTEX_Y,Lb_ENDVERTEX_Z,Lb_M,Lb_MM,L_ENDVERTEX_X,L_ENDVERTEX_Y,L_ENDVERTEX_Z,L_M,L_MM,...,pim_PP_POSMOMCOV_1_1,pim_PP_POSMOMCOV_1_3,pim_PP_POSMOMCOV_1_4,pim_PP_POSMOMCOV_1_5,pim_PP_POSMOMCOV_3_3,pim_PP_POSMOMCOV_3_4,pim_PP_POSMOMCOV_3_5,pim_PP_POSMOMCOV_4_4,pim_PP_POSMOMCOV_4_5,pim_PP_POSMOMCOV_5_5
0,1.674756,0.053261,0.004662,5551.938396,5989.289405,461.662234,-78.219730,6499.551889,1099.307456,1116.740451,...,0.156104,0.054083,-1.250532,1.336292,135.629715,-9.136084,3142.555664,17.418783,-213.811951,7.316263e+04
1,0.880395,-0.130328,15.654318,4900.777975,5194.460691,209.723720,199.205588,4382.077506,1227.863044,1286.303104,...,1.845794,-4.761426,-4.757078,-16.119181,2562.330811,9.229958,9031.324219,43.331825,32.646484,3.185937e+04
2,0.893070,-0.210214,27.346291,5666.157904,5456.847848,85.100993,-116.538278,5570.951003,1201.393959,1207.424049,...,0.156974,-0.272185,-2.498910,9.883920,760.612305,745.142334,-28860.775391,787.374390,-28372.277344,1.099216e+06
3,0.636035,-0.090822,-65.799339,5664.613011,5525.276190,-207.708523,152.682518,7298.531631,1142.063975,1140.580639,...,0.171479,0.277545,-2.941218,-7.591522,503.966400,-231.546097,-13094.367188,201.608795,6041.502441,3.405917e+05
4,0.986496,-0.173897,54.366702,5455.855712,5341.861787,133.445419,3.864480,6608.471141,1103.601064,1113.048444,...,0.174056,0.364528,-2.574241,10.254116,495.785950,39.231781,13094.487305,70.691956,1017.335205,3.464277e+05
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4615,0.847599,-0.210711,4.450656,5692.657914,5654.795686,223.345177,-152.925186,4344.444760,1218.763592,1210.425120,...,4.293598,-41.613556,-8.433111,-125.821960,10386.176758,-773.569397,31718.019531,105.008934,-2364.728271,9.691066e+04
4616,1.244354,0.273509,26.617187,5793.378536,6047.448576,303.079116,143.467671,6927.816528,1161.330798,1176.832557,...,1.251937,-2.488792,-19.226019,-123.327477,909.107849,1038.543579,42415.023438,1501.186279,48571.347656,1.983485e+06
4617,0.771458,-0.198550,-45.870119,9427.587168,9146.296960,-114.674239,-98.523163,5355.674345,5155.192814,2493.913859,...,0.975354,-11.960934,-1.553255,54.025459,13881.547852,-4576.853027,-64122.265625,1564.207886,21157.076172,2.963137e+05
4618,0.893271,-0.240819,-8.364338,5179.496251,5229.171622,-104.381881,-225.769432,4373.238772,840.462502,1165.228188,...,1.932934,-4.882315,-2.684560,7.398481,2144.610107,197.797516,-3460.504395,26.012659,-318.839905,5.589844e+03


In [16]:
df_Z = import_data(input_Z)
df_Z

,Lb_ENDVERTEX_X,Lb_ENDVERTEX_Y,Lb_ENDVERTEX_Z,Lb_M,Lb_MM,L_ENDVERTEX_X,L_ENDVERTEX_Y,L_ENDVERTEX_Z,L_M,L_MM,...,pim_PP_POSMOMCOV_1_1,pim_PP_POSMOMCOV_1_3,pim_PP_POSMOMCOV_1_4,pim_PP_POSMOMCOV_1_5,pim_PP_POSMOMCOV_3_3,pim_PP_POSMOMCOV_3_4,pim_PP_POSMOMCOV_3_5,pim_PP_POSMOMCOV_4_4,pim_PP_POSMOMCOV_4_5,pim_PP_POSMOMCOV_5_5
0,0.880396,-0.130330,15.654310,4861.049567,5199.267583,209.591476,199.310411,4382.276209,1229.188474,1287.379829,...,1.845794,-4.761426,-4.757078,-16.119181,2562.330811,9.229958,9031.324219,43.331825,32.646484,3.185937e+04
1,0.893068,-0.210206,27.346308,5654.994619,5444.895215,84.919736,-116.122610,5553.225945,1195.764524,1203.850079,...,0.156974,-0.272185,-2.498910,9.883920,760.612305,745.142334,-28860.775391,787.374390,-28372.277344,1.099216e+06
2,0.605712,-0.203043,22.148030,5669.144965,5617.693874,-204.696740,18.912832,3801.145534,1196.170905,1139.619726,...,0.158962,-0.209039,-1.985822,-4.321545,1088.073242,209.480865,26939.328125,90.186363,5198.505859,6.694764e+05
3,0.986431,-0.173929,54.368627,5432.497035,5344.981925,133.559090,3.908184,6613.524449,1103.084458,1113.033732,...,0.174056,0.364528,-2.574241,10.254116,495.785950,39.231781,13094.487305,70.691956,1017.335205,3.464277e+05
4,0.964621,0.025102,-0.666843,-3450.057763,8147.558111,-149.922147,185.041983,3091.428388,1020.048337,1415.006565,...,3.806791,37.160542,-7.632927,92.395973,75195.250000,18212.148438,191457.765625,4514.575684,46385.707031,4.875446e+05
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4116,0.734721,-0.098737,-61.234970,5689.378695,5644.035102,-114.013384,-30.097036,4114.596593,1254.867358,1283.669391,...,0.333853,-4.867212,-3.991302,36.106815,12834.206055,900.435059,-98631.953125,126.835609,-6909.498047,7.582912e+05
4117,0.847599,-0.210711,4.450658,5700.898251,5662.319975,224.137084,-153.416377,4357.518693,1227.039954,1215.676639,...,4.293598,-41.613556,-8.433111,-125.821960,10386.176758,-773.569397,31718.019531,105.008934,-2364.728271,9.691066e+04
4118,1.244335,0.273425,26.616683,5830.374068,6115.096462,303.306922,143.556052,6933.471508,1169.752481,1178.300654,...,1.251937,-2.488792,-19.226019,-123.327477,909.107849,1038.543579,42415.023438,1501.186279,48571.347656,1.983485e+06
4119,0.771442,-0.198652,-45.869861,9510.956119,9192.143023,-114.707922,-98.384425,5365.467329,5204.126939,2498.918252,...,0.975354,-11.960934,-1.553255,54.025459,13881.547852,-4576.853027,-64122.265625,1564.207886,21157.076172,2.963137e+05


## Merge DataFrames

In [17]:
merge_columns = [
    'MCTRUTH_L_ENDVERTEX_X',
    'MCTRUTH_L_ENDVERTEX_Y',
    'MCTRUTH_L_ENDVERTEX_Z',
    'MCTRUTH_Lb_ENDVERTEX_X',
    'MCTRUTH_Lb_ENDVERTEX_Y',
    'MCTRUTH_Lb_ENDVERTEX_Z',
    'MCTRUTH_p_PX',
    'MCTRUTH_p_PY',
    'MCTRUTH_p_PZ',
    'MCTRUTH_pim_PX',
    'MCTRUTH_pim_PY',
    'MCTRUTH_pim_PZ',
    'MCTRUTH_mum_PX',
    'MCTRUTH_mum_PY',
    'MCTRUTH_mum_PZ',
    'MCTRUTH_mup_PX',
    'MCTRUTH_mup_PY',
    'MCTRUTH_mup_PZ',
    'MCTRUTH_L_ID',
    'MCTRUTH_Lb_ID',
]

In [18]:
dic_VF = {}
dic_SIGMAX = {}
dic_SIGMAY = {}
dic_SIGMAZ = {}

for element in merge_columns:
    dic_VF[element + '_VF'] = element
    dic_SIGMAX[element + '_SIGMAX'] = element
    dic_SIGMAY[element + '_SIGMAY'] = element
    dic_SIGMAZ[element + '_SIGMAZ'] = element

In [19]:
df_VF = df_VF.add_suffix('_VF')
df_VF.rename(columns=dic_VF, inplace=True)

df_X = df_X.add_suffix('_SIGMAX')
df_X.rename(columns=dic_SIGMAX, inplace=True)

df_Y = df_Y.add_suffix('_SIGMAY')
df_Y.rename(columns=dic_SIGMAY, inplace=True)

df_Z = df_Z.add_suffix('_SIGMAZ')
df_Z.rename(columns=dic_SIGMAZ, inplace=True)

Some rows are reconstructed from the same MCTRUTH events. They are not many, but the merger does not like when this happens.

In [20]:
df_VF.drop_duplicates(subset=merge_columns, inplace=True)
df_X.drop_duplicates(subset=merge_columns, inplace=True)
df_Y.drop_duplicates(subset=merge_columns, inplace=True)
df_Z.drop_duplicates(subset=merge_columns, inplace=True)

In [21]:
df_X['L_ALGO_ID_SIGMAX'].value_counts()

Generic2DAlgorithm    4576
Name: L_ALGO_ID_SIGMAX, dtype: int64

In [22]:
df_tmp = df_VF.merge(df_X, how='outer', on=merge_columns, suffixes=[None, None])
df_tmp = df_tmp.merge(df_Y, how='outer', on=merge_columns, suffixes=[None, None])
df_events = df_tmp.merge(df_Z, how='outer', on=merge_columns, suffixes=[None, None])

del(df_tmp)

In [23]:
df_events

,Lb_ENDVERTEX_X_VF,Lb_ENDVERTEX_Y_VF,Lb_ENDVERTEX_Z_VF,Lb_M_VF,Lb_MM_VF,L_ENDVERTEX_X_VF,L_ENDVERTEX_Y_VF,L_ENDVERTEX_Z_VF,L_M_VF,L_MM_VF,...,pim_PP_POSMOMCOV_1_1_SIGMAZ,pim_PP_POSMOMCOV_1_3_SIGMAZ,pim_PP_POSMOMCOV_1_4_SIGMAZ,pim_PP_POSMOMCOV_1_5_SIGMAZ,pim_PP_POSMOMCOV_3_3_SIGMAZ,pim_PP_POSMOMCOV_3_4_SIGMAZ,pim_PP_POSMOMCOV_3_5_SIGMAZ,pim_PP_POSMOMCOV_4_4_SIGMAZ,pim_PP_POSMOMCOV_4_5_SIGMAZ,pim_PP_POSMOMCOV_5_5_SIGMAZ
0,1.674753,0.053259,0.004623,5548.595304,5994.525949,461.983261,-78.247981,6504.222043,1098.518302,1116.608101,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0.880395,-0.130333,15.654301,4900.959051,5196.066164,209.545799,199.243538,4381.083975,1228.240710,1286.308889,...,1.845794,-4.761426,-4.757078,-16.119181,2.562331e+03,9.229958e+00,9.031324e+03,43.331825,3.264648e+01,3.185937e+04
2,0.893069,-0.210212,27.346294,5669.609293,5447.484208,84.953310,-116.199707,5557.096303,1196.546077,1204.210206,...,0.156974,-0.272185,-2.498910,9.883920,7.606123e+02,7.451423e+02,-2.886078e+04,787.374390,-2.837228e+04,1.099216e+06
3,0.636031,-0.090822,-65.799311,5666.024183,5523.351851,-207.686722,152.674077,7297.794399,1142.059990,1140.502451,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0.986496,-0.173897,54.366690,5458.097962,5344.372839,133.523208,3.904319,6612.102919,1103.525428,1113.284255,...,0.174056,0.364528,-2.574241,10.254116,4.957859e+02,3.923178e+01,1.309449e+04,70.691956,1.017335e+03,3.464277e+05
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5755,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.551270,-6.912379,-7.934309,-26.857990,1.071044e+04,3.091554e+03,4.294379e+04,928.774475,1.239755e+04,1.722678e+05
5756,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.237656,-24.787832,-10.313980,-185.614258,2.536955e+04,-2.565136e+03,1.932977e+05,452.399902,-1.958405e+04,1.473124e+06
5757,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.928140,-3.058603,-5.626563,83.982750,1.018535e+03,-1.053006e+03,-2.851886e+04,1230.861694,2.961616e+04,8.017929e+05
5758,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.193072,-180.229996,-93.571724,4815.377441,4.230150e+06,1.814929e+06,-1.133648e+08,781405.812500,-4.863967e+07,3.038179e+09


# Performance cross-comparison

In [24]:
converges_with_VF = ~df_events['L_ALGO_ID_VF'].isna()
converges_with_X = ~df_events['L_ALGO_ID_SIGMAX'].isna()
converges_with_Y = ~df_events['L_ALGO_ID_SIGMAY'].isna()
converges_with_Z = ~df_events['L_ALGO_ID_SIGMAZ'].isna()

In [25]:
def statistics_increase(condition: pd.Series):
    return sum(condition) / sum(converges_with_VF)

In [26]:
print("Baseline of reco events:\t", sum(converges_with_VF))
print("Sigma_x stats increase:\t\t", "{:.2%}".format(statistics_increase(converges_with_X & ~converges_with_VF)))
print("Sigma_y stats increase:\t\t", "{:.2%}".format(statistics_increase(converges_with_Y & ~converges_with_VF)))
print("Sigma_z stats increase:\t\t", "{:.2%}".format(statistics_increase(converges_with_Z & ~converges_with_VF)))

Baseline of reco events:	 4561
Sigma_x stats increase:		 14.97%
Sigma_y stats increase:		 18.88%
Sigma_z stats increase:		 20.72%


In [27]:
def condition_over_total(condition: pd.Series):
    return sum(condition) / len(df_events)

In [28]:
print("Events exclusive to sigma_x (over total):\t", "{:.2%}".format(condition_over_total(~converges_with_VF & converges_with_X & ~converges_with_Y & ~converges_with_Z)))
print("Events exclusive to sigma_y (over total):\t", "{:.2%}".format(condition_over_total(~converges_with_VF & ~converges_with_X & converges_with_Y & ~converges_with_Z)))
print("Events exclusive to sigma_z (over total):\t", "{:.2%}".format(condition_over_total(~converges_with_VF & ~converges_with_X & ~converges_with_Y & converges_with_Z)))

Events exclusive to sigma_x (over total):	 1.49%
Events exclusive to sigma_y (over total):	 2.14%
Events exclusive to sigma_z (over total):	 1.01%


In [29]:
def condition_over_recovered(condition: pd.Series):
    return sum(condition) / sum(~converges_with_VF)

In [30]:
print("Events exclusive to sigma_x (over recovered):\t", "{:.2%}".format(condition_over_recovered(~converges_with_VF & converges_with_X & ~converges_with_Y & ~converges_with_Z)))
print("Events exclusive to sigma_y (over recovered):\t", "{:.2%}".format(condition_over_recovered(~converges_with_VF & ~converges_with_X & converges_with_Y & ~converges_with_Z)))
print("Events exclusive to sigma_z (over recovered):\t", "{:.2%}".format(condition_over_recovered(~converges_with_VF & ~converges_with_X & ~converges_with_Y & converges_with_Z)))

Events exclusive to sigma_x (over recovered):	 7.17%
Events exclusive to sigma_y (over recovered):	 10.26%
Events exclusive to sigma_z (over recovered):	 4.84%


# Performance on subsets of events

In [31]:
def print_median_performance(
    algorithm: str,
    series: pd.Series,
    series_name: str,
    output_format: str
) -> None:
    print(f"{algorithm} median {series_name}:\t{output_format.format(series.median())}")

In [32]:
def performance_on_subset(condition: pd.Series, algorithm: str) -> None:
    print_median_performance(algorithm,
                             df_events.loc[condition, f'L_VFASPF_CHI2_VDOF_{algorithm}'],
                             'chi2_vdof', '{:.2f}')
    print_median_performance(algorithm,
                             (df_events[f'L_ENDVERTEX_Z_{algorithm}'] - df_events['MCTRUTH_L_ENDVERTEX_Z']).loc[condition],
                             'Lz vtx bias', '{:.2f}')
    print_median_performance(algorithm,
                             ((df_events[f'DTF_FixJPsiLambda_p_PZ_{algorithm}'] - df_events['MCTRUTH_p_PZ'])/df_events['MCTRUTH_p_PZ']).loc[condition],
                             'pz(p) DTF bias', '{:.3%}')

In [33]:
def algo_performance_comparison(
    condition: pd.Series,
    algos: List[str] = ['VF', 'SIGMAX', 'SIGMAY', 'SIGMAZ']
) -> None:
    
    for algo in algos:
        performance_on_subset(condition, algo)
        print()

## On vertex fitter events

In [34]:
algo_performance_comparison(converges_with_VF)

VF median chi2_vdof:	1.02
VF median Lz vtx bias:	396.40
VF median pz(p) DTF bias:	0.032%

SIGMAX median chi2_vdof:	1.09
SIGMAX median Lz vtx bias:	495.04
SIGMAX median pz(p) DTF bias:	0.099%

SIGMAY median chi2_vdof:	1.32
SIGMAY median Lz vtx bias:	545.82
SIGMAY median pz(p) DTF bias:	0.106%

SIGMAZ median chi2_vdof:	1.45
SIGMAZ median Lz vtx bias:	648.30
SIGMAZ median pz(p) DTF bias:	0.193%



## On $\sigma_x$-rescaled converging events

In [35]:
algo_performance_comparison(converges_with_X)

VF median chi2_vdof:	1.06
VF median Lz vtx bias:	483.76
VF median pz(p) DTF bias:	0.077%

SIGMAX median chi2_vdof:	1.48
SIGMAX median Lz vtx bias:	507.27
SIGMAX median pz(p) DTF bias:	0.047%

SIGMAY median chi2_vdof:	1.63
SIGMAY median Lz vtx bias:	619.82
SIGMAY median pz(p) DTF bias:	0.114%

SIGMAZ median chi2_vdof:	1.88
SIGMAZ median Lz vtx bias:	640.34
SIGMAZ median pz(p) DTF bias:	0.153%



## On $\sigma_y$-rescaled converging events

In [36]:
algo_performance_comparison(converges_with_Y)

VF median chi2_vdof:	1.28
VF median Lz vtx bias:	526.40
VF median pz(p) DTF bias:	0.045%

SIGMAX median chi2_vdof:	1.61
SIGMAX median Lz vtx bias:	604.44
SIGMAX median pz(p) DTF bias:	0.106%

SIGMAY median chi2_vdof:	1.81
SIGMAY median Lz vtx bias:	550.81
SIGMAY median pz(p) DTF bias:	-0.002%

SIGMAZ median chi2_vdof:	2.01
SIGMAZ median Lz vtx bias:	654.74
SIGMAZ median pz(p) DTF bias:	0.093%



## On $\sigma_z$-rescaled converging events

In [37]:
algo_performance_comparison(converges_with_Z)

VF median chi2_vdof:	1.36
VF median Lz vtx bias:	615.43
VF median pz(p) DTF bias:	0.138%

SIGMAX median chi2_vdof:	1.84
SIGMAX median Lz vtx bias:	621.38
SIGMAX median pz(p) DTF bias:	0.105%

SIGMAY median chi2_vdof:	1.96
SIGMAY median Lz vtx bias:	648.13
SIGMAY median pz(p) DTF bias:	0.098%

SIGMAZ median chi2_vdof:	2.38
SIGMAZ median Lz vtx bias:	663.99
SIGMAZ median pz(p) DTF bias:	0.071%



## On common events

In [38]:
algo_performance_comparison(converges_with_VF & converges_with_X & converges_with_Y & converges_with_Z)

VF median chi2_vdof:	1.35
VF median Lz vtx bias:	628.34
VF median pz(p) DTF bias:	0.164%

SIGMAX median chi2_vdof:	1.36
SIGMAX median Lz vtx bias:	639.38
SIGMAX median pz(p) DTF bias:	0.163%

SIGMAY median chi2_vdof:	1.38
SIGMAY median Lz vtx bias:	646.94
SIGMAY median pz(p) DTF bias:	0.203%

SIGMAZ median chi2_vdof:	1.41
SIGMAZ median Lz vtx bias:	651.99
SIGMAZ median pz(p) DTF bias:	0.203%



# On exclusive events

In [39]:
algo_performance_comparison(converges_with_VF & ~converges_with_X & ~converges_with_Y & ~converges_with_Z, algos=['VF'])

VF median chi2_vdof:	0.32
VF median Lz vtx bias:	-294.60
VF median pz(p) DTF bias:	1.168%



In [40]:
algo_performance_comparison(~converges_with_VF & converges_with_X & ~converges_with_Y & ~converges_with_Z, algos=['SIGMAX'])

SIGMAX median chi2_vdof:	3.18
SIGMAX median Lz vtx bias:	518.04
SIGMAX median pz(p) DTF bias:	-0.408%



In [41]:
algo_performance_comparison(~converges_with_VF & ~converges_with_X & converges_with_Y & ~converges_with_Z, algos=['SIGMAY'])

SIGMAY median chi2_vdof:	3.13
SIGMAY median Lz vtx bias:	245.81
SIGMAY median pz(p) DTF bias:	-5.373%



In [42]:
algo_performance_comparison(~converges_with_VF & ~converges_with_X & ~converges_with_Y & converges_with_Z, algos=['SIGMAZ'])

SIGMAZ median chi2_vdof:	51.52
SIGMAZ median Lz vtx bias:	1547.15
SIGMAZ median pz(p) DTF bias:	-1.009%



## On recovered events (for each algorithm)

In [43]:
algo_performance_comparison(~converges_with_VF & converges_with_X, algos=['SIGMAX'])

SIGMAX median chi2_vdof:	6.14
SIGMAX median Lz vtx bias:	541.54
SIGMAX median pz(p) DTF bias:	-0.483%



In [44]:
algo_performance_comparison(~converges_with_VF & converges_with_Y, algos=['SIGMAY'])

SIGMAY median chi2_vdof:	5.76
SIGMAY median Lz vtx bias:	572.95
SIGMAY median pz(p) DTF bias:	-1.167%



In [45]:
algo_performance_comparison(~converges_with_VF & converges_with_Z, algos=['SIGMAZ'])

SIGMAZ median chi2_vdof:	9.17
SIGMAZ median Lz vtx bias:	729.39
SIGMAZ median pz(p) DTF bias:	-0.710%

